# Residual Networks

<p style="text-align:justify"> Welcome to the second exercise of this CNN Tutorial! This exercise is similar to the second assignment of week 2 in the Deep Learning Specialization - course 4 by Professor Andrew Ng [https://www.coursera.org/learn/convolutional-neural-networks]. But we will implement in PyTorch. We will build a very deep CNN based on the ResNet architecture. This was introduced by [He et al.](https://arxiv.org/pdf/1512.03385.pdf). Generally  deep nets are very difficult to train but ResNet allow you to train much deeper networks than were previously practically feasible.

**In this exercise, we will:**
- Implement the basic building blocks of ResNets. 
- Put together these building blocks to implement and train a state-of-the-art neural network for image classification. 

Before jumping into the problem, let's run the cell below to load the required packages.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, sampler

import numpy as np
import matplotlib.pyplot as plt
import h5py
import time

from resnet_utils import *

%matplotlib inline

# Understanding ResNet
<p style="text-align:justify"> A very plain deep network constructed by stacking more layers to a shallower net suffers from vanishing gradient problem (or exploding problem) resulting in a surprisingly higher training error and test error. Figure 1 below depicts this on CIFAR-10 dataset.

<img src="images/plain_deep_net_problem1.png" style="width:450px;height:220px;">
<caption><center> <u> <font color='purple'> **Figure 1** </u><font color='purple'>  : **Less Deep vs Plain deep** <br> Train and Test error for a plain deeper network (right figure) higher than the lesser deep network (left figure) </center></caption>

<p style="text-align:justify"> ResNet overcomes this problem by introducing identity connections. By construction, a deep net adds layers to a shallower net. If the added layers have identity mapping then the training error of deep net can be expected to be no more than that of the shallower net. A layer in a plain net learns $H(x) = f(x)$ while in ResNet it learns  $H(x) = f(x) + identity(x) = f(x) + x$. $f$ is called as the residual mapping with respect to identity. See Figure 2 for a pictorial depiction.

<img src="images/plainNet_resNet.png" style="width:450px;height:220px;">
<caption><center> <u> <font color='purple'> **Figure 2** </u><font color='purple'>  : **Plain Net vs ResNet** <br> ResNet with the identity connection also called skip connection </center></caption>


# Coding ResNet50
<p style="text-align:justify"> Let us start coding ResNet50, a residual net with 50 layers deep. As dicussed in our earlier presentation, ResNet is made up of Identity blocks and Convolutional blocks. Let's build Identity block.

## Identity Block

<p style = "textalign:justify"> The identity block is the standard block used in ResNets, and corresponds to the case where the input activation at layer $l$ has the same dimension as the output activation at layer $l+s$ for some s > 1. We will implement the following identity block where the skip connection skips over three hidden layers i.e s = 3. The upper path is the "shortcut path." The lower path is the "main path." Batchnorm is added to speed up training. See Figure 3. 

<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **Figure 3** </u><font color='purple'>  : **Identity block.** Skip connection "skips over" 3 layers.</center></caption>

We will complete the class IdentityBlock and the forward method for the class in the following cell.

Here're the details of parameters associted with various sub-blocks in the "main" and the "shortcut" path with respect to the class IdentityBlock we are building in the following cell.

Let out_channels be the triple ($F_1$, $F_2$, $F_3$) where $F_i$ is the number of filters required for ith CONV2D sub-block in Figure 3. For eg, $F_1$ is out_channels[0].

First component of main path: 
- The first CONV2D called as *conv1* has $F_1$ filters with shape (1,1) and a stride of (1,1). It has zero padding - i.e "valid" convolution. [Hint:nn.Conv2d](http://pytorch.org/docs/master/nn.html#conv2d)
- The first BatchNorm called as *bn1* is then performed. [Hint: nn.BatchNorm2d](http://pytorch.org/docs/master/nn.html#batchnorm2d)
- Then apply the ReLU activation function called as *relu1*. 
[Hint: nn.ReLU2d](http://pytorch.org/docs/master/nn.html#relu)

Second component of main path:
- The second CONV2D called as *conv2* has $F_2$ filters of shape $(f,f)$ and a stride of (1, 1). The padding should be such that resulting convolution is "same" convolution. Depending on $f$ this could be uniform in all directions or non-uniform. For this exercise, we can safely take padding to be (1, 1) since $f$ is going to be 3.
- The second BatchNorm called as *bn2* is then performed.   
- Then apply the ReLU activation function called as *relu2*. 

Third component of main path:
- The third CONV2D called as *conv3* has $F_3$ filters of shape (1,1) and a stride of (1,1).  It has zero padding - i.e "valid" convolution. 
- The third BatchNorm called as *bn3* is then performed. Note that there is no ReLU activation function in this component. 

Since "shortcut" involves only identity mapping and does not involve any convolutions we will directly deal with it in 'forward' method of the class.

For the 'forward' method:
- save the input $x$ in $x\_shortcut$ as it is required for combining with the output of the main path
- Forward propagate input $x$ through the first, second and third component of the main path
- Add output of main path and $x\_shortcut$ and apply ReLU use to this aggregate. For applying ReLU here, you can use relu method in [torch.nn.functional](http://pytorch.org/docs/master/nn.html#id18) which is imported with name F in this notebook i.e you can call F.relu <br>

<b> Fill Code in the following cell only where you are asked to fill. Leave other lines as it is.

In [2]:
class IdentityBlock(nn.Module):
    
    def __init__(self, f, in_channels, out_channels):
        """
        Implementation of the class IdentityBlock as defined in Figure 3

        Arguments:
        f -- integer, specifying the shape of the middle CONV's window for the main path
        in_channels -- integer, specifying the number of incoming channels to this IdentityBlock
        out_channels -- a triple where ith component is number of filters for the ith CONV2D 
                        sub-block in the main path
        """
        super().__init__() 

        self.f = f
        self.in_channels = in_channels
        self.out_channels = out_channels

        #Replace "None" in the right hand sides (rhs) of following with your code. Each rhs is one line of code. 
        #Description and hints above may be helpful

        self.conv1 = None
        self.bn1 = None
        self.relu1 = None
        self.conv2 = None
        self.bn2 = None
        self.relu2 = None
        self.conv3 = None
        self.bn3 =  None 
    
    def forward(self, x):
        """
        Forward propagation through IdentityBlock

        Arguments:
        x -- the input, a 4D Tensor of dimension (batch_size x in_channels x height x width)

        Returns:
        a 4D Tensor of dimension (batch_size x output_channels[2] x height x width)    
        """
        #Replace "None" in the right hand sides (rhs) of following with your code. 
        #Few of them already filled to help you.
        #Each rhs is one line of code. 
        #Description and hints above may be helpful
        
        x_shortcut = None #save x
        x = self.conv1(x)
        x = None # apply first batch normalization
        x = self.relu1(x)
        x = None # apply second convolution
        x = self.bn2(x)
        x = None # apply second relu activation
        x = None # apply 3rd convolution
        x = None # apply 3rd batch normalization
        x = F.relu(x + x_shortcut)
        return x

In [ ]:
#Run this cell to test the code and see if you get the expected output
torch.manual_seed(23)
idb = IdentityBlock(3, 1, (1, 3, 1))
x = torch.randn(1, 1, 3, 3)
output = idb(Variable(x))
print(output)

<b> Expected Output:</b>
<br>
<span style="color:green">
<br>
Variable containing:
<br>
(0 ,0 ,.,.) = 
<br>
  0.0000  1.2414  0.0405
  <br>
  0.0000  0.0000  0.5557
  <br>
  1.2214  0.0000  0.8336
  <br>
[torch.FloatTensor of size 1x1x3x3]</span>

## Convolutional Block
<p style = "textalign:justify"> This type of block is used in the ResNet when the input and output dimensions don't match up. The difference with the identity block is that there is a CONV2D layer in the shortcut path - see Figure 4. 
<img src="images/convblock_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **Figure 4** </u><font color='purple'>  : **Convolutional block** </center></caption>

The CONV2D layer in the shortcut path is used to resize the input $x$ to a different dimension, so that the dimensions match up in the final addition needed to add the shortcut value back to the main path. For example, to reduce the activation dimensions's height and width by a factor of 2, you can use a 1x1 convolution with a stride of 2. The CONV2D layer on the shortcut path does not use any non-linear activation function. Its main role is to just apply a (learned) linear function that reduces the dimension of the input, so that the dimensions match up for the later addition step. 

We will complete the class ConvBlock and the 'forward' method for the class in the following cell.

Here are the details of parameters associted with various sub-blocks in the "main" and the "shortcut" path with respect to the class ConvBlock we are building in the following cell.

Let out_channels be the triple ($F_1$, $F_2$, $F_3$) where $F_i$ is the number of filters required for ith CONV2D sub-block in Figure 4. For eg, $F_1$ is out_channels[0].

First component of main path: 
- The first CONV2D called as *conv1* has $F_1$ filters with shape (1,1) and a stride of (s,s). It has zero padding - i.e "valid" convolution. [Hint:nn.Conv2d](http://pytorch.org/docs/master/nn.html#conv2d)
- The first BatchNorm called as *bn1* is then performed. [Hint: nn.BatchNorm2d](http://pytorch.org/docs/master/nn.html#batchnorm2d)
- Then apply the ReLU activation function called as *relu1*. 
[Hint: nn.ReLU2d](http://pytorch.org/docs/master/nn.html#relu)

Second component of main path:
- The second CONV2D called as *conv2* has $F_2$ filters of shape $(f,f)$ and a stride of (1, 1). The padding should be such that resulting convolution is "same" convolution. Depending on $f$ this could be uniform in all directions or non-uniform. For this exercise, we can safely take padding to be (1, 1) since $f$ is going to be 3.
- The second BatchNorm called as *bn2* is then performed.   
- Then apply the ReLU activation function called as *relu2*. 

Third component of main path:
- The third CONV2D called as *conv3* has $F_3$ filters of shape (1,1) and a stride of (1,1).  It has zero padding - i.e "valid" convolution. 
- The third BatchNorm called as *bn3* is then performed. Note that there is no ReLU activation function in this component. 

Shortcut path:
- The CONV2D called as *conv_shortcut* has $F_3$ filters of shape (1,1) and a stride of (s,s). It has zero padding - i.e "valid" convolution.
- BatchNorm called as *bn_shortcut* is then performed.   

For the 'forward' method:
- save the input $x$ in $x\_shortcut$ as it is required for combining with the output of the main path
- Forward propagate input $x$ through the first, second and third component of the main path
- Forward propagate the input (saved in $x\_shortcut$) through convolution and batch normalization layers of shortcut   path
- Add output of main path and $x\_shortcut$ and apply ReLU to this aggregate. For applying ReLU here, you can use relu method in [torch.nn.functional](http://pytorch.org/docs/master/nn.html#id18) which is imported with name F in this notebook i.e you can call F.relu <br>

<b> Fill Code in the following cell only where you are asked to fill. Leave other lines as it is.

In [3]:
class ConvBlock(nn.Module):
    
    def __init__(self, f, in_channels, out_channels, s = 2):
        """
        Implementation of the class ConvBlock as defined in Figure 4

        Arguments:
        f -- integer, specifying the shape of the middle CONV's window for the main path
        in_channels -- integer, specifying the number of incoming channels to this ConvBlock
        out_channels -- a triple where ith component is number of filters for the ith CONV2D 
                        sub-block in the main path
        s -- integer specifying the stride parameter required for first convolution and 
             convolution for shortcut path. Default is set to 2.
        """
        super().__init__()
        
        self.f = f
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.s = s
        
        # Replace "None" in the right hand sides (rhs) of following with your code. Each rhs is one line of code. 
        # Description and hints above may be helpful
        
        self.conv1 = None
        self.bn1 = None
        self.relu1 = None
        self.conv2 = None
        self.bn2 = None
        self.relu2 = None
        self.conv3 = None
        self.bn3 = None
        self.relu3 = None
        self.conv_shortcut = None
        self.bn_shortcut = None
        
    def forward(self, x):
        """
        Forward propagation through ConvBlock

        Arguments:
        x -- the input, a 4D Tensor of dimension (batch_size x in_channels x height x width)

        Returns:
        a 4D Tensor of dimension (batch_size x output_channels[2] x height x width)    
        """
        
        #Replace "None" in the right hand sides (rhs) of following with your code. 
        #Few of them already filled to help you.
        #Each rhs is one line of code. 
        #Description and hints above may be helpful
        
        x_shortcut = None #save x
        x = None # apply 1st convolution
        x = None # apply 1st batch normalization
        x = None # apply 1st relu activation
        x = None # apply 2nd convolution
        x = None # apply 2nd batch normalization
        x = None # apply 2nd relu activation
        x = None # apply 3rd convolution
        x = None # apply 3rd batch normalization
        x_shortcut = None # apply convolution for shortcut path
        x_shortcut = None # apply batch normalization for shortcut path
        x = None # apply relu activation to aggregate from main and shortcut paths. Use F.relu
        return x
        

In [ ]:
#Run this cell to test the code and see if you get the expected output
torch.manual_seed(23)
cvb = ConvBlock(3, 3, (1, 3, 2), s = 2)
x = torch.randn(1, 3, 3, 3)
output = cvb(Variable(x))
print(output)

<b> Expected Output:</b>
<br>
<span style="color:green">
<br>
Variable containing:
<br>
(0 ,0 ,.,.) = 
<br>
  0.0000  0.7471
  <br>
  0.9794  0.0000
  <br>
  <br>
(0 ,1 ,.,.) = 
<br>
  1.4802  0.0000
  <br>
  0.3219  0.0000
  <br>
[torch.FloatTensor of size 1x2x2x2]</span>


## The ResNet50 class
<p style = "textaligh:justify">With basic blocks ready in our kitty, let us build the ResNet50 class. Figure 5 describes in detail ResNet50 architecture. "ID BLOCK" in the diagram stands for "Identity block," and "ID BLOCK x3" means that 3 identity blocks are to be stacked together. Similar notations follow for convolutional block (CONVBLOCK).

<img src="images/resnet_kiank.png" style="width:850px;height:150px;">
<caption><center> <u> <font color='purple'> **Figure 5** </u><font color='purple'>  : **ResNet-50 model** </center></caption>

The details of this ResNet-50 model are:
#### Stage1
- A convolution layer called as *conv_stg1* has 64 filters with shape (7, 7) and a stride of (2, 2). It has (3, 3)       padding  [Hint:nn.Conv2d](http://pytorch.org/docs/master/nn.html#conv2d)
- A BatchNorm called as *bn_stg1* is then performed. 
[Hint: nn.BatchNorm2d](http://pytorch.org/docs/master/nn.html#batchnorm2d)
- A max poling layer called as *max_pool_stg1* with kernel_size = (3, 3) and stride = (2, 2). 
[Hint: nn.MaxPool2d](http://pytorch.org/docs/master/nn.html#maxpool2d)

#### Stage2
- A ConvBlock layer named *convblk1_stg2* with parameters $f = 3, \, in\_channels = 64, \, out\_channels = (64, 64, 256), \, s = 1$
- Two IdentityBlock's named *idblk1_stg2* and *idblk2_stg2* respectively with parameters $f = 3, \, in\_channels = 256, \, out\_channels = (64, 64, 256)$

#### Stage 3
- A ConvBlock layer named *convblk1_stg3* with parameters $f = 3, \, in\_channels = 256, \, out\_channels = (128, 128, 512), \, s = 2$
- Three IdentityBlock's named *idblk1_stg3*, *idblk2_stg3* and *idblk3_stg3* respectively with parameters $f = 3, \, in\_channels = 512, \, out\_channels = (128, 128, 512)$

#### Stage 4
- A ConvBlock layer named *convblk1_stg4* with parameters $f = 3, \, in\_channels = 512, \, out\_channels = (256, 256, 1024), \, s = 2$
- Five IdentityBlock's named *idblk1_stg4*, *idblk2_stg4*, *idblk3_stg4*, *idblk4_stg4* and *idblk5_stg4* respectively with parameters $f = 3, \, in\_channels = 1024, \, out\_channels = (256, 256, 1024)$

#### Stage 5
- A ConvBlock layer named *convblk1_stg5* with parameters $f = 3, \, in\_channels = 1024, \, out\_channels = (512, 512, 2048), \, s = 2$
- Two IdentityBlock's named *idblk1_stg5* and *idblk2_stg5* respectively with parameters $f = 3, \, in\_channels = 2048, \, out\_channels = (512, 512, 2048)$

#### Pooling and Linear Layer
- An average pooling layer called *avg_pool* with kernel_size = (2, 2). 
[Hint: nn.AvgPool2D](http://pytorch.org/docs/master/nn.html#avgpool2d)
- A fully connected layer (also called linear layer) named *fc* with parameters (2048, 6). The six here stands for the number of classes in the classification problem that we will try out below with this ResNet50. To create a linear layer, use [nn.Linear](http://pytorch.org/docs/master/nn.html#linear)

<b> Fill Code in the following cell only where you are asked to fill. Leave other lines as it is. In fact you are required to fill only stages 3 and four. Rest including 'forward' method done for want of time. Also initialization of weight and bias parameters are also done as part of __init__ method. But definitely go through the entire class.


In [4]:
class ResNet50(nn.Module):
    
    def __init__(self):
        
        super().__init__()
           
        self.conv_stg1 = nn.Conv2d(3, 64, kernel_size = (7, 7), stride = (2, 2), padding = (3, 3))
        self.bn_stg1 = nn.BatchNorm2d(64)
        self.max_pool_stg1 = nn.MaxPool2d(kernel_size = (3, 3), stride = (2, 2))
        
        self.convblk1_stg2 = ConvBlock(3, 64, (64, 64, 256), s = 1)
        self.idblk1_stg2 = IdentityBlock(3, 256, (64, 64, 256))
        self.idblk2_stg2 = IdentityBlock(3, 256, (64, 64, 256))
        
        self.convblk1_stg3 = None
        self.idblk1_stg3 = None
        self.idblk2_stg3 = None
        self.idblk3_stg3 = None
        
        self.convblk1_stg4 = None
        self.idblk1_stg4 = None
        self.idblk2_stg4 = None
        self.idblk3_stg4 = None
        self.idblk4_stg4 = None
        self.idblk5_stg4 = None
        
        self.convblk1_stg5 = ConvBlock(3, 1024, (512, 512, 2048), s = 2)
        self.idblk1_stg5 = IdentityBlock(3, 2048, (512, 512, 2048))
        self.idblk2_stg5 = IdentityBlock(3, 2048, (512, 512, 2048))
        
        self.avg_pool = nn.AvgPool2d(kernel_size = (2, 2))
        self.fc =  nn.Linear(2048, 6)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform(m.weight.data)                
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()               
       
        
    def forward(self, x):
        x = self.conv_stg1(x)
        x = self.bn_stg1(x)
        x = self.max_pool_stg1(x)
        x = self.convblk1_stg2(x)
        x = self.idblk1_stg2(x)
        x = self.idblk2_stg2(x)
        x = self.convblk1_stg3(x)
        x = self.idblk1_stg3(x)
        x = self.idblk2_stg3(x)
        x = self.idblk3_stg3(x)
        x = self.convblk1_stg4(x)
        x = self.idblk1_stg4(x)
        x = self.idblk2_stg4(x)
        x = self.idblk3_stg4(x)
        x = self.idblk4_stg4(x)
        x = self.idblk5_stg4(x)
        x = self.convblk1_stg5(x)
        x = self.idblk1_stg5(x)
        x = self.idblk2_stg5(x)
        x = self.avg_pool(x)
        x = x.view(-1, 2048)        
        x = self.fc(x)
        return x

## Load, Build, Train and Test ResNet50 Model
<p style = textalign:justify> Let us now load a dataset, build the ResNet50 model, train it and test it.
We will be using Signs dataset [2]. It is a collection of 6 signs representing numbers from 0 to 5 - see Figure 6.
<img src="images/SIGNS.png" style="width:450px;height:150px;">
<caption><center> <u> <font color='purple'> **Figure 6** </u><font color='purple'>  : **SIGNS dataset** </center></caption>
<br>
Each image is RGB 64 x 64. There are 1080 training images and 120 test images.

In [6]:
# Run this cell to load training data
train_data_loader, train_data_size = load_dataset('datasets/train_signs.h5', transform = None)

<p style = "textalign:justify"> We have to instantiate ResNet50 class - i.e build the model, then train it for some epochs and test it. We will use cross entropy loss [nn.CrossEntropyLoss](http://pytorch.org/docs/master/nn.html#crossentropyloss) as our loss function and Adam optimizer [torch.optim.Adam](http://pytorch.org/docs/master/optim.html#algorithms) for back propogation. Here, you will train only for 2 epochs in order to check if you have coded ResNet50 class correctly. This will take 3 to 4 minutes in your system.Then you will use a pretrained model trained earlier by us for testing. We trained the model on Titan-x GPU for 1000 epochs. The error after 1000 epochs was below $10^{-8}$. It took around 30 minutes to train for us. 

In [7]:
# Build and train ResNet50 model
# Replace rhs of line 3 with cross entropy loss with default parameters
# Replace rhs of line 4 with Adam optimizer with default parameters
torch.manual_seed(23)
resnet50 = ResNet50()
criterion = None
optimizer = None
resnet50.num_epochs = 2
# Do not uncomment the two lines below.
#if torch.cuda.is_available():
#    resnet50.cuda()
train_resnet50(resnet50, train_data_loader, train_data_size, criterion, optimizer)

<b> Expected Output:</b>
<br>
<span style="color:green">
Epoch 0/1
<br>\----------
<br>Train Loss: should be between 1.5 and 1.8
<br>Epoch 1/1
<br>\----------
<br>Train Loss: should be between 0.55 and 0.7
<br>Training completes in between 3 to 5 minutes</span>

In [8]:
# Run this cell to test the model
model = ResNet50()
predict(model, 'datasets/test_signs.h5', 'resnet50.pth')

<b> Expected Output:</b>
<br>
<span style="color:green">
Accuracy of the network on the 120 test images: 96 %</span>

In [1]:
# Run this cell to get  a summary of the model
summary = model_summary((3, 64, 64), resnet50)
print(summary)

<span style="color:purple">
Congratulations! Having completed this exercise and the previous one, you should now be comfortable with basic CNN and a powerful architecture namely ResNet. Also you should have some minimum idea about using PyTorch to code CNN's. We have more to go and hopefully it will be beneficial to you all. Looking forward to you all in the next session. Sairam.
</span>

### References 

1. Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun - [Deep Residual Learning for Image Recognition (2015)](https://arxiv.org/abs/1512.03385)

2. [http://kaiminghe.com/icml16tutorial/icml2016_tutorial_deep_residual_networks_kaiminghe.pdf]

3. [https://www.coursera.org/learn/convolutional-neural-networks]